# Agent State Machine - Predicate

## Example

In this example, we will demonstrate a **Predicate** state whose sole existence is to decide if the output is **true** or **false**


```mermaid
stateDiagram-v2
INIT --> PREDICATE: next / action
PREDICATE --> TRUE(): next / action / condition_true
    TRUE() --> FINAL: next / action
PREDICATE --> FALSE(): next / action / condition_false
    FALSE() --> FINAL: next / action
```

Ask if the sky is blue

In [1]:
from gai.asm import AsyncStateMachine

# Define the predicate: ask the LLM if the sky is blue

def sky_is_blue(state) -> bool:
    from gai.llm.openai import OpenAI,boolean_schema
    client = OpenAI(client_config={
        "type": "ttt",
        "client_type": "openai",
    })
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[{"role": "user", "content": "The sky is blue."}],
        response_format=boolean_schema, 
        max_tokens=100,
        temperature=0.0
    )
    import json
    content = json.loads(response.choices[0].message.content)  # Parse the JSON response
    return bool(content["result"])

# Define the predicate actions

async def run_if_true(state):
    print("Action: THE ANSWER IS TRUE")

async def run_if_false(state):
    print("Action: THE ANSWER IS FALSE")

with AsyncStateMachine.StateMachineBuilder("""
    INIT --> PREDICATE
    PREDICATE --> TRUE: condition_true
        TRUE --> FINAL 
    PREDICATE --> FALSE: condition_false
        FALSE --> FINAL    
    """) as builder:
        fsm = builder.build({
                "PREDICATE": {
                    "module_path": "gai.asm.states",
                    "class_name": "PurePredicateState",
                    "title": "PREDICATE",
                    "predicate": "sky_is_blue",
                    "output_data": ["predicate_result"],
                    "conditions": ["condition_true", "condition_false"]
                },
                "TRUE": {
                    "module_path": "gai.asm.states",
                    "class_name": "PureActionState",
                    "input_data": {
                        "title": "TRUE"
                    },
                    "action": "run_if_true"
                },
                "FALSE": {
                    "module_path": "gai.asm.states",
                    "class_name": "PureActionState",
                    "input_data": {
                        "title": "FALSE"
                    },
                    "action": "run_if_false"
                }
            },
            sky_is_blue=sky_is_blue,
            run_if_true=run_if_true,
            run_if_false=run_if_false
            )

In [2]:
fsm.restart()
await fsm.run_async()

# state output:
print("Current state is", fsm.state)
print("Result is",fsm.state_bag["predicate_result"])


Current state is PREDICATE
Result is True


In [3]:
await fsm.run_async()

# state output:
print("Current state is", fsm.state)


Action: THE ANSWER IS TRUE
Current state is TRUE
